In [ ]:
import numpy as np
from tqdm import tqdm
import time

np.random.seed(1)

In [ ]:
def sigmoid(x): return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x): return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
N   = 4            # số mẫu huấn luyện
x_1 = [0, 0, 1, 1] # đầu vào thứ nhất
x_2 = [0, 1, 0, 1] # đầu vào thứ hai
y   = [0, 1, 1, 0] # đầu ra

In [ ]:
'''
np.random.normal(0, 1): lấy một giá trị ngẫu nhiên từ phân phối chuẩn
có giá trị trung bình là 0 và độ lệch chuẩn là 1
'''

w, dw = {}, {}

for j in range(1, 2 + 1): # chỉ số nơ-ron j của lớp thứ nhất
    for k in range(0, 2 + 1): # chỉ k = 0 để chỉ bias
        w[f'{j}{k}_1'] = np.random.normal(0, 1) # w_jk_r trong đó r = 1
        dw[f'{j}{k}_1'] = 0 #
    
for k in range(0, 2 + 1): # chỉ k = 0 để chỉ bias
    w[f'1{k}_2'] = np.random.normal(0, 1) # w_jk_r trong đó r = 2
    dw[f'1{k}_2'] = 0 #
    
# trong thực tế, các thư viện sử dụng một số kỹ thuật khác để khởi tạo trọng số

In [ ]:
mu = 1 # tốc độ học
nb_epochs = 200 # số lần lặp

In [ ]:
# huấn luyện
# trong thực tế, các thư viện sử dụng kỹ thuật phân rã tốc độ học, khiến nó giảm dần theo thời gian
pbar = tqdm(range(nb_epochs))
for epoch in pbar:
    J, nb_correct = 0, 0
    
    for i in range(N):
        # tính toán forward cho lớp ẩn thứ nhất
        v_1_1 = w['10_1'] + x_1[i] * w['11_1'] + x_2[i] * w['12_1']
        v_2_1 = w['20_1'] + x_1[i] * w['21_1'] + x_2[i] * w['22_1']
        y_1_1 = sigmoid(v_1_1)
        y_2_1 = sigmoid(v_2_1)
        
        # tính toán forward cho lớp đầu ra
        v_1_2 = w['10_2'] + y_1_1 * w['11_2'] + y_2_1 * w['12_2']
        y_1_2 = sigmoid(v_1_2)
                      
        # Cách 1: tính toán backward cho lớp đầu ra dựa trên hàm lỗi bình phương
        # delta_1_2 = (y_1_2 - y[i]) * sigmoid_derivative(v_1_2)
        # J += 0.5 * (y_1_2 - y[i]) ** 2
        
        # Cách 2: tính toán backward cho lớp đầu ra dựa trên hàm lỗi binary cross-entropy
        delta_1_2 = y_1_2 - 1 if y[i] else y_1_2
        J += -np.log(y_1_2) if y[i] else -np.log(1 - y_1_2)
        
        nb_correct += ((y_1_2 > 0.5) == y[i])
                
        # tính toán backward cho lớp ẩn thứ nhất
        delta_1_1 = (delta_1_2 * w['11_2']) * sigmoid_derivative(v_1_1)
        delta_2_1 = (delta_1_2 * w['12_2']) * sigmoid_derivative(v_2_1)
        
        # tính toán các cập nhật trọng số
        dw['10_2'] += delta_1_2 * 1
        dw['11_2'] += delta_1_2 * y_1_1
        dw['12_2'] += delta_1_2 * y_2_1
        
        dw['10_1'] += delta_1_1 * 1
        dw['11_1'] += delta_1_1 * x_1[i]
        dw['12_1'] += delta_1_1 * x_2[i]
        
        dw['20_1'] += delta_2_1 * 1
        dw['21_1'] += delta_2_1 * x_1[i]
        dw['22_1'] += delta_2_1 * x_2[i]
        
    pbar.set_postfix({'cost_value': J, 'accuracy': str(100*nb_correct//N) + '%'})
    time.sleep(0.05)
    if nb_correct == N: break
                    
    # cập nhật trọng số của lớp thứ nhất
    for j in range(1, 2 + 1):
        for k in range(0, 2 + 1): # chỉ k = 0 để chỉ bias
            w[f'{j}{k}_1'] += -mu * dw[f'{j}{k}_1'] # tốc độ học * ước lượng trọng số thay đổi
            dw[f'{j}{k}_1'] = 0 # reset gradient, zero grad
    
    # cập nhật trọng số của lớp thứ hai
    for k in range(0, 2 + 1): # chỉ k = 0 để chỉ bias
        w[f'1{k}_2'] += -mu * dw[f'1{k}_2'] # tốc độ học * ước lượng trọng số thay đổi
        dw[f'1{k}_2'] = 0# reset gradient, zero grad

In [ ]:
# dự đoán
for i in range(N):
    # tính toán forward cho lớp ẩn thứ nhất
    v_1_1 = w['10_1'] + x_1[i] * w['11_1'] + x_2[i] * w['12_1']
    v_2_1 = w['20_1'] + x_1[i] * w['21_1'] + x_2[i] * w['22_1']
    y_1_1 = sigmoid(v_1_1)
    y_2_1 = sigmoid(v_2_1)

    # tính toán forward cho lớp đầu ra
    v_1_2 = w['10_2'] + y_1_1 * w['11_2'] + y_2_1 * w['12_2']    
    y_1_2 = sigmoid(v_1_2)
    
    print(x_1[i], x_2[i], y_1_2 > 0.5)